In [1]:
import pandas as pd
import numpy as np
import gutenbergpy.textget
import re
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from plotnine import *
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import utils

In [2]:
# Load in datasets)
book_contents_train = utils.load_book_contents(utils.book_authors_train)
book_contents_test = utils.load_book_contents(utils.book_authors_test)

In [3]:
# Tokenize the data
books_train_wtoks = utils.wtok_books(book_contents_train)
books_test_wtoks = utils.wtok_books(book_contents_test)

In [4]:
# Get 100 samples per book of around 1000 words each
from importlib import reload
reload(utils)
book_samples_train = utils.get_samples(books_train_wtoks, 100, [1000, 1000], random_seed=42)
book_samples_test = utils.get_samples(books_test_wtoks, 100, [1000, 1000], random_seed=42)

In [5]:
# Do feature engineering
# Use ngram frequency as features
# cd_1grams is the frequency of 1-grams associated with Charles Dickens, for example
data_df_train = utils.get_data_df(book_samples_train, utils.book_authors_train)
data_df_test = utils.get_data_df(book_samples_test, utils.book_authors_test)

In [6]:
tgt_cols = data_df_test.columns
tgt_cols = ['author_name']
X_train = data_df_train.drop(tgt_cols,axis=1)
y_train = data_df_train.filter(tgt_cols).to_numpy().ravel()
X_test = data_df_test.drop(tgt_cols,axis=1)
y_test = data_df_test.filter(tgt_cols).to_numpy().ravel()

In [7]:
print(X_train.shape)

(1800, 9)


In [8]:
gb_model =  GradientBoostingClassifier()
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
conf_matrix = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred))
conf_matrix.index = set(y_train)
conf_matrix.columns = conf_matrix.index
display(conf_matrix)
print(metrics.classification_report(y_test, y_pred))
conf_matrix

Accuracy:  0.834


,Charles Dickens,Herman Melville,Jane Austen
Charles Dickens,254,26,20
Herman Melville,83,402,15
Jane Austen,4,18,178


                 precision    recall  f1-score   support

Charles Dickens       0.74      0.85      0.79       300
Herman Melville       0.90      0.80      0.85       500
    Jane Austen       0.84      0.89      0.86       200

       accuracy                           0.83      1000
      macro avg       0.83      0.85      0.83      1000
   weighted avg       0.84      0.83      0.84      1000



,Charles Dickens,Herman Melville,Jane Austen
Charles Dickens,254,26,20
Herman Melville,83,402,15
Jane Austen,4,18,178


In [9]:
cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5)
cv_scores

array([0.82222222, 0.91111111, 0.89444444, 0.88888889, 0.92222222])

In [10]:
gb_model.feature_importances_

array([0.15198319, 0.16766473, 0.00509395, 0.07296784, 0.06482246,
       0.24648268, 0.1291404 , 0.13359851, 0.02824624])

In [11]:
pd.DataFrame(dict([(k, pd.Series(v)) for k, v in utils.book_authors_test.items()]))

,Charles Dickens,Herman Melville,Jane Austen
0,786.0,4045,946.0
1,580.0,8118,1212.0
2,883.0,2694,NaN
3,NaN,13720,NaN
4,NaN,53861,NaN


In [12]:
def get_author_num(data, sample_id):
    sample_id = int(str(sample_id).split("_")[0])

    authors_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
    author_name = pd.melt(authors_df).query("value == @sample_id").variable.tolist()[0]
    return ['Charles Dickens', 'Jane Austen','Herman Melville'].index(author_name)
train_data = [(words, get_author_num(utils.book_authors_train, sample_id)) for sample_id, words in book_samples_train.items()]
test_data = [(words, get_author_num(utils.book_authors_test, sample_id)) for sample_id, words in book_samples_test.items()]

In [13]:
from collections import Counter

all_words = [word for words, _ in train_data for word in words]
word_counts = Counter(all_words)
vocab = {word: idx + 1 for idx, (word, count) in enumerate(word_counts.most_common(1000))}

vocab['<PAD>'] = 0

In [14]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        words, label = self.data[idx]
        word_indices = [self.vocab.get(word, 0) for word in words]
        return torch.tensor(word_indices), torch.tensor(int(label))  # Assuming labels are integers

In [15]:
import torch.nn as nn

class AuthorClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(AuthorClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])  # Take the last LSTM output for each sequence
        return output

In [16]:
len(train_data[2][0])

1000

In [17]:
from torch.utils.data import DataLoader
import torch

torch.manual_seed(42)

# Hyperparameters
embedding_dim = 64
hidden_size = 128
num_classes = len(set(label for _, label in train_data))
vocab_size = len(vocab)

# Model, loss, optimizer
model = AuthorClassifier(vocab_size, embedding_dim, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_dataset = TextDataset(train_data, vocab)
test_dataset = TextDataset(test_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [40]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    print(epoch)
    model.train()
    loss_vals = []
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        loss_vals.append(loss.item())
        optimizer.step()
    print(np.mean(loss_vals))

0
0.00017544011681916585
1
0.1742499962016601
2
0.033068424989387654
3
0.009168187295201054
4
0.005444631658559828
5
0.003443440687924362
6
0.002506110321918274
7
0.002193269828128171
8
0.0012795981515011047
9
0.0011520669703665924
10
0.000890846585287108
11
0.0007625546277267858
12
0.0006901539046227475
13
0.0005863397236680612
14
0.0005403648486989958
15
0.00048335105167464923
16
0.0004396393557902213
17
0.00040006375906829675
18
0.0003647578893210984
19
0.000334019190695631
20
0.00031997596166196703
21
0.0002849184326662094
22
0.00026490130799254727
23
0.014991197815744957
24
0.0060001259864708245
25
0.00268905136845046
26
0.0007841484937735164
27
0.0005353668366616036
28
0.0004003508234873471
29
0.0003844800563743583
30
0.09275942282029223
31
0.03424599888048282
32
0.005242597149366415
33
0.0022354954623886826
34
0.0016951613024024194
35
0.0011267703651499591
36
0.0008770188315681656
37
0.0006811675572718837
38
0.0005368015616815619
39
0.0004691875717406602
40
0.0004164311474979561

In [36]:
torch.save(model.state_dict, "weights.pth")

In [23]:
model.load_state_dict(torch.load("weights.pth")())

<All keys matched successfully>

In [24]:
# Evaluation
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 81.00%
